In [1]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

In [2]:
#%reload_ext autoreload

In [3]:
data_root = '/Volumes/Samsung_T5/MIT/mta'
derived_data = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
#os.listdir(data_root)


In [4]:
baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}
version_form = ['version_form']

qsts = ['snap', 'ssrs',  'pc', 'wechsler'] #masc to many missing data 

In [5]:
#outcome variablles 
snap_vars = ['snainatx', 'snahypax', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['ssptossx', 'sspintx']# social skills mean, internalizing mean 
#masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars,  'pc': pc_vars, 'wechsler': wechsler_vars}


In [6]:
interaction_predictors = ['days_baseline', 'site', 'trtname'] #time, site, treatment group

# mediator variables
comorb_mediators  = ['cdorodd' , 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa'] #ODD/CD or anx excluding specific phobia 
services_mediators =  ['demo61'] #reciept of public assistance 
prev_med_mediators = ['hi_24'] #medication intake prior to study 

#moderator variables 
accept_moderator = ['d2dresp'] # initail acceptance of treatment 

In [7]:
# load files, drop rows if missing date, drop duplicates 

snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
#masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group = 'treatment_groups.csv'

snap_ = pd.read_csv(Path(data_root, snap_file), delimiter="\t", usecols=np.concatenate((baseline_var, snap_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
ssrs_ = pd.read_csv(Path(data_root, ssrs_file), delimiter="\t", usecols=np.concatenate((baseline_var, ssrs_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
#masc_ = pd.read_csv(Path(data_root, masc_file), delimiter="\t", usecols=np.concatenate((baseline_var, masc_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
pc_ = pd.read_csv(Path(data_root, parent_child_file), delimiter="\t", usecols=np.concatenate((baseline_var, pc_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
wechsler_ = pd.read_csv(Path(data_root, wechsler_file), delimiter="\t", usecols=np.concatenate((baseline_var, wechsler_vars)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
treat_group = pd.read_csv(Path(derived_data, treat_group))

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_37481/2167685383.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  snap_ = pd.read_csv(Path(data_root, snap_file), delimiter="\t", usecols=np.concatenate((baseline_var, snap_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_37481/2167685383.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ssrs_ = pd.read_csv(Path(data_root, ssrs_file), delimiter="\t", usecols=np.concatenate((baseline_var, ssrs_vars, version_form)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates()
/var/folders/h

In [8]:
# merge with treatment group info convert colum data to appropriate dtypes 

snap = prep.set_baseline_dtypes(pd.merge(snap_, treat_group, how='inner', on = 'src_subject_id')).dropna()#.table with relevant snap vales, rater, and treatment group 
ssrs = prep.set_baseline_dtypes(pd.merge(ssrs_, treat_group, how='inner', on = 'src_subject_id').dropna())#.dropna() #table with relevant snap vales, rater, and treatment group 
#masc = prep.set_baseline_dtypes(pd.merge(masc_, treat_group, how='inner', on = 'src_subject_id')).dropna()#.dropna() #table with relevant snap vales, rater, and treatment group 
pc = prep.set_baseline_dtypes(pd.merge(pc_, treat_group, how='inner', on = 'src_subject_id').dropna())#.dropna() #table with relevant snap vales, rater, and treatment group 
wechsler = prep.set_baseline_dtypes(pd.merge(wechsler_, treat_group, how='inner', on = 'src_subject_id')).dropna()#.dropna() #table with relevant snap vales, rater, and treatment group 

print(snap_.shape, ssrs.shape, pc.shape, wechsler.shape)

Success
Success
Success
Success
(14540, 10) (10735, 10) (8627, 10) (4571, 10)


In [9]:
ssrs.loc[ssrs['version_form'].str.startswith('Teacher'), 'version_form'] = 'Teacher'
ssrs.loc[ssrs['version_form'].str.startswith('Parent'), 'version_form'] = 'Parent'

In [10]:
comorb_file = 'diagpsx01.txt' # contains odd/cd and anx comorbid diagnoses 
demog_file = 'demgr01.txt' # contains recieot of public assistance 
health_qst_file  = 'health01.txt' # contains previous medication 
init_sat_file = 'debrief01.txt' # contains rating of initial acceptance of treatment goup

comorb = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, comorb_file), delimiter= '\t', usecols = np.concatenate((baseline_var, comorb_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
demog = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, demog_file), delimiter= '\t', usecols  =np.concatenate((baseline_var, services_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
health_qst = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, health_qst_file), delimiter= '\t', usecols=np.concatenate((baseline_var, prev_med_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
init_sat_file = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, init_sat_file), delimiter= '\t', usecols=np.concatenate((baseline_var,  accept_moderator)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())



Success
Success
Success
Success


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_37481/3879433410.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  comorb = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, comorb_file), delimiter= '\t', usecols = np.concatenate((baseline_var, comorb_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='days_baseline').drop_duplicates())
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_37481/3879433410.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  demog = prep.set_baseline_dtypes(pd.read_csv(Path(data_root, demog_file), delimiter= '\t', usecols  =np.concatenate((baseline_var, services_mediators)), skiprows=[1] , parse_dates=['days_baseline']).dropna(subset='d

In [11]:
snap_split_dict = prep.split_data_from_timepoints(snap)
ssrs_split_dict = prep.split_data_from_timepoints(ssrs)
pc_split_dict = prep.split_data_from_timepoints(pc)
wechsler_split_dict = prep.split_data_from_timepoints(wechsler)
data = {'snap' : snap_split_dict, 'ssrs': ssrs_split_dict, 'pc': pc_split_dict, 'wechsler': wechsler_split_dict}

In [12]:
pc_split_dict['14']['days_baseline'].value_counts()

days_baseline
0      2880
419      36
196      31
392      30
448      30
       ... 
49        1
63        1
308       1
307       1
327       1
Name: count, Length: 437, dtype: int64

In [13]:
data['pc']['14']['days_baseline'] == pc_split_dict['14']['days_baseline']

0       True
1       True
2       True
3       True
4       True
        ... 
8665    True
8666    True
8667    True
8668    True
8669    True
Name: days_baseline, Length: 6985, dtype: bool

In [14]:
data['pc']['14']['days_baseline'].value_counts()

days_baseline
0      2880
419      36
196      31
392      30
448      30
       ... 
49        1
63        1
308       1
307       1
327       1
Name: count, Length: 437, dtype: int64

In [33]:
gen_interact_formula = 'C(trtname, Treatment(reference = "L")) * days_baseline * C(site)' # reapeat with log days 
formulas =  [[' ~ '.join((var, gen_interact_formula)) for var in values] for values in outcomes_dict.values()]
formulas_dict = dict(zip(outcomes_dict.keys(), formulas))
formulas_dict

{'snap': ['snainatx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'snahypax ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'snaoddx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)'],
 'ssrs': ['ssptossx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'sspintx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)'],
 'pc': ['pcrcpax ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'pcrcprx ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)'],
 'wechsler': ['w1readb ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'w2math ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)',
  'w3spell ~ C(trtname, Treatment(reference = "L")) * days_baseline * C(site)']}

In [34]:

groups = 'src_subject_id'
alpha = 0.05
hyps_interactions = var_dict.get_hyps_interactions()

In [35]:
# result, summ, hsmm = rr.get_RR_stats(formula, data, groups=groups, alpha= alpha)
# rr.f_test_interactions(result, hyps_interactions, alpha)

In [36]:
raters = ['Parent', 'Teacher']

In [37]:
result_snap = [[ smf.mixedlm(formulas_dict['snap'][i], data['snap']['14'][data['snap']['14']['version_form'] == rater], groups = groups).fit() for rater in raters] for i in range(len(snap_vars))]

In [38]:
result_ssrs = [[ smf.mixedlm(formulas_dict['ssrs'][i], data['ssrs']['14'][data['ssrs']['14']['version_form'] == rater], groups = groups).fit() for rater in raters] for i in range(len(ssrs_vars))]

In [39]:
#test = data['pc']['14'][data['pc']['14']['version_form'] == 'Parent'].copy()

In [40]:
#scaler = StandardScaler()

In [41]:
#test['days_baseline'] = scaler.fit_transform(np.array(test['days_baseline']).reshape(-1, 1))

In [42]:
#test_pc_formula = 'pcrcpax ~ C(trtname, Treatment(reference = "L"))* days_baseline  '

In [43]:
#result = smf.mixedlm(formulas_dict['pc'][0], data['pc']['14'][data['pc']['14']['version_form'] == 'Parent'], groups = groups).fit() 

In [44]:
#result.summary()

In [45]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# X = result.model.exog
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
# vif["features"] = result.model.exog_names
# print(vif)

In [46]:
#result_pc2 =  smf.mixedlm(formulas_dict['pc'][1], data['pc']['14'][data['pc']['14']['version_form'] == 'Parent'], groups = groups).fit()  

In [47]:
result_wechlser = [ smf.mixedlm(formulas_dict['wechsler'][i], data['wechsler']['14'], groups = groups).fit()  for i in range(len(wechsler_vars))]

In [48]:
## worked for snap, ssrs, wechsler

In [49]:
hyps_interactions = var_dict.get_hyps_interactions()
hyps_interactions

{'site': 'C(site)[T.2] = C(site)[T.3] = C(site)[T.4] = C(site)[T.5] = C(site)[T.6] = 0',
 'site_treat': 'C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.2] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.3] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.4] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.5] = C(trtname, Treatment(reference="L"))[T.M]:C(site)[T.6] = C(trtname, Treatment(reference="L"))[T.P]:C(site)[T.6] = C(trtname, Treatment(reference="L"))[T.C]:C(site)[T.6] = 0',
 'time_treat': 'C(trtname, Treatment

In [50]:
for i, var in enumerate(snap_vars):
    for res in result_snap[i]:
        print(var, rr.f_test_interactions(res, hyps_interactions, alpha))

snainatx   Description     Significance    F-Value       P-Value
0        site  Not Significant   0.478532  7.925425e-01
1  site_treat  Not Significant   0.778523  7.031027e-01
2  time_treat    *Significant*  11.422714  1.850028e-07
snainatx   Description     Significance    F-Value       P-Value
0        site  Not Significant   0.285858  9.210400e-01
1  site_treat  Not Significant   1.474889  1.054912e-01
2  time_treat    *Significant*  16.061269  2.349415e-10
snahypax   Description     Significance    F-Value       P-Value
0        site  Not Significant   0.111711  9.898156e-01
1  site_treat  Not Significant   1.643687  5.523849e-02
2  time_treat    *Significant*  14.544289  1.998723e-09
snahypax   Description     Significance    F-Value       P-Value
0        site  Not Significant   1.069212  3.753642e-01
1  site_treat  Not Significant   1.527963  8.670372e-02
2  time_treat    *Significant*  21.565982  8.193047e-14
snaoddx   Description     Significance   F-Value   P-Value
0        

In [51]:
for i, var in enumerate(ssrs_vars):
    for res in result_ssrs[i]:
        print(var, rr.f_test_interactions(res, hyps_interactions, alpha))

ssptossx   Description     Significance   F-Value   P-Value
0        site  Not Significant  0.970836  0.434083
1  site_treat  Not Significant  0.935707  0.522976
2  time_treat  Not Significant  0.861553  0.460281
ssptossx   Description     Significance   F-Value   P-Value
0        site  Not Significant  0.485249  0.787516
1  site_treat  Not Significant  0.496734  0.943616
2  time_treat    *Significant*  5.101886  0.001608
sspintx   Description     Significance   F-Value   P-Value
0        site  Not Significant  0.631833  0.675473
1  site_treat  Not Significant  0.771165  0.711268
2  time_treat  Not Significant  1.776329  0.149427
sspintx   Description     Significance   F-Value   P-Value
0        site  Not Significant  0.368243  0.870591
1  site_treat  Not Significant  0.456635  0.961461
2  time_treat  Not Significant  0.100833  0.959549


In [59]:
for res in result_wechlser:
    print(res)

w1readb   Description     Significance   F-Value   P-Value
0        site    *Significant*  2.736320  0.018097
1  site_treat  Not Significant  0.935301  0.523636
2  time_treat  Not Significant  0.635075  0.592424
w1readb   Description     Significance   F-Value   P-Value
0        site  Not Significant  1.924660  0.087290
1  site_treat  Not Significant  1.141942  0.312511
2  time_treat  Not Significant  0.376829  0.769732
w1readb   Description     Significance   F-Value   P-Value
0        site  Not Significant  1.516170  0.181586
1  site_treat  Not Significant  0.609549  0.869312
2  time_treat  Not Significant  0.060611  0.980460
w2math   Description     Significance   F-Value   P-Value
0        site    *Significant*  2.736320  0.018097
1  site_treat  Not Significant  0.935301  0.523636
2  time_treat  Not Significant  0.635075  0.592424
w2math   Description     Significance   F-Value   P-Value
0        site  Not Significant  1.924660  0.087290
1  site_treat  Not Significant  1.141942  0.